In [1]:
# Import dependencies
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from google.colab import files
uploaded = files.upload()

Saving All_Data.csv to All_Data.csv


Random Forest Regressor - function

In [3]:
def rfr_machine(X, y) : 
    # Divide data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Train random forest algorithm to solve this regression problem
    regressor = RandomForestRegressor(n_estimators=50, random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Prepare cleaned data for machine

In [4]:
# Pull in working dataset
all_data_df = pd.read_csv("All_Data.csv")

In [5]:
# Check for null values
all_data_df.isnull().sum()

FIPS                                            0
Num_Per_100k                                    7
State                                           0
County                                          0
Percent_Smokers                                 0
Percent_Obese                                   0
Food_Environment_Index                         19
Percent_Access_Exercise                         6
Primary_Care_Physician_Rate                   134
Mental_Health_Providers_Rate                  264
Percent_Some_College                            0
Percent_Unemployed                              1
Income_Inequality_Ratio                         2
Violent_Crime_Rate                            176
Average_Daily_Air_Polution_Particle_Matter     34
Percent_Long_Commute_Drives_Alone               0
Percent_Rural                                   7
dtype: int64

In [6]:
# Drop null values
all_data_df = all_data_df.dropna(axis=0)

In [7]:
# Drop identifier columns
all_data_usable_df = all_data_df.drop(["FIPS", "State", "County"], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [8]:
# https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/
# Divide data into attributes and labels

X = all_data_usable_df.drop(["Num_Per_100k"], 1)
y = all_data_usable_df["Num_Per_100k"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [9]:
# Call function to run machine

rfr_machine(X, y)

Mean Absolute Error: 9.381482041587901
Mean Squared Error: 147.05343147069945
Root Mean Squared Error: 12.12655892950261


Random Forest Regressor - health features

In [10]:
# Drop non-health feature columns
health_features_df = all_data_usable_df.drop(["Percent_Rural", "Percent_Long_Commute_Drives_Alone", "Average_Daily_Air_Polution_Particle_Matter", "Violent_Crime_Rate", "Income_Inequality_Ratio", "Percent_Unemployed", "Percent_Some_College"], 1)
health_features_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Num_Per_100k,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Access_Exercise,Primary_Care_Physician_Rate,Mental_Health_Providers_Rate
0,107.0,19.0,36.0,7.1,71.0,45.0,13.0
1,82.5,17.0,29.0,7.9,69.0,73.0,89.0
2,96.1,22.0,44.0,5.5,53.0,42.0,8.0
3,112.3,20.0,38.0,7.6,49.0,53.0,9.0
4,96.1,20.0,36.0,8.5,32.0,21.0,9.0


In [11]:
# Divide into attributes and labels
X = all_data_usable_df.iloc[:, 1:7].values
y = all_data_usable_df.iloc[:, 0].values

In [12]:
# Call function to run machine on just health features

rfr_machine(X, y)

Mean Absolute Error: 10.262215500945178
Mean Squared Error: 172.17535106238185
Root Mean Squared Error: 13.121560542190927


Random Forest Regressor - environment / economic features

In [16]:
ee_features_df = all_data_usable_df.drop(["Percent_Smokers", "Percent_Obese", "Food_Environment_Index", "Percent_Access_Exercise", "Primary_Care_Physician_Rate", "Mental_Health_Providers_Rate"], 1)
ee_features_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Num_Per_100k,Percent_Some_College,Percent_Unemployed,Income_Inequality_Ratio,Violent_Crime_Rate,Average_Daily_Air_Polution_Particle_Matter,Percent_Long_Commute_Drives_Alone,Percent_Rural
0,107.0,61.9,5.3,4.4,266.0,10.8,41.0,42.0
1,82.5,63.8,5.4,4.6,217.0,9.6,40.0,42.3
2,96.1,39.7,8.6,5.9,329.0,10.1,35.0,67.8
3,112.3,49.8,6.6,4.2,147.0,10.5,49.0,68.4
4,96.1,53.9,5.5,4.1,212.0,11.3,60.0,90.0


In [14]:
# Divide into attributes and labels
X = all_data_usable_df.iloc[:, 1:6].values
y = all_data_usable_df.iloc[:, 0].values

In [15]:
# Call function to run machine on just environment / economic features

rfr_machine(X, y)

Mean Absolute Error: 10.329909262759925
Mean Squared Error: 174.4235131115312
Root Mean Squared Error: 13.20694942488731
